In [ ]:
# https://stackoverflow.com/questions/55313610/importerror-libgl-so-1-cannot-open-shared-object-file-no-such-file-or-directo
!apt-get update && apt-get install libgl1

***Эксперимент 1***

*Сравнение каждого лица из датасета со всеми остальными. 
Цель - оценить количество ошибок первого рода (т.е. случаев, когда дистанция между эмбедингами изображений лиц двух разных людей меньше порога для отдельно взятой модели).*

Результаты:

1. Таблица с дистанциями для пар лиц;
2. Матрицы ошибок для каждой из моделей;
3. Диаграммы рассеяния дистанций для каждого человека для каждой модели;

In [ ]:
!pip install deepface -q
!pip install matplotlib -q

In [34]:
import cv2
from PIL import Image, ImageOps
import time
import os
import glob
from deepface import DeepFace
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import seaborn as sns

In [44]:
folder_path = '/tf/data/testt/center/center'
tmp_folder = "new_tmp"
os.makedirs(tmp_folder, exist_ok=True)

In [38]:
model_names = [
    "VGG-Face",
    "Facenet",
    "Facenet512",
    "OpenFace",
    "DeepFace",
    "DeepID",
    "ArcFace",
    "SFace"
]

In [39]:
model_resolutions = {
    "VGG-Face": 224,
    "Facenet": 220,
    "Facenet512": 160,
    "OpenFace": 96,
    "DeepFace": 152,
    "DeepID": 55,
    "ArcFace": 112,
    "SFace": 112,
}

In [40]:
def crop_to_square_pil(image, face_info, border_pixels=0):
    
    x, y, w, h = face_info['facial_area']['x'], face_info['facial_area']['y'], face_info['facial_area']['w'], face_info['facial_area']['h']
    square_size = max(w, h) + 2 * border_pixels
    new_x = max(0, x)
    new_y = max(0, y)
    cropped_image = image.crop((new_x - (h-w)//2 - border_pixels, new_y - border_pixels, new_x - (h-w)//2 + square_size, new_y + square_size))

    return cropped_image

In [41]:
def resize_image_pil(image, new_resolution):
    resized_img = image.resize(new_resolution, Image.LANCZOS)
    return resized_img

In [20]:
!export XLA_FLAGS=--xla_gpu_cuda_data_dir=/usr/local/cuda

In [21]:
!ls /usr/local/

README.md  cuda     cuda-11.8  games	lib  package_info.json	sbin   src
bin	   cuda-11  etc        include	man  requirements.txt	share


In [45]:
for model_name in model_names:
    resolution = model_resolutions[model_name]
    df_list = []

    files = glob.glob(os.path.join(folder_path, "**", "*.JPG"), recursive=True)
    # print(files)
    for img_path in tqdm(files, desc=f'Creating images cropped&resized to {resolution} for {model_name}', unit='image'):
        rel_path = os.path.relpath(img_path, folder_path)
        rel_path = rel_path.replace(os.path.sep, "_")
        output_path = os.path.join(tmp_folder, f"{rel_path[:-4]}_{resolution}.jpg")
        # print(f"{output_path}")
        if not os.path.exists(output_path):
            image = Image.open(img_path)
            image = ImageOps.exif_transpose(image)
            # display(image)
            faces = DeepFace.extract_faces(img_path=img_path, detector_backend="retinaface")
            for face_info in faces:
                cropped_face = crop_to_square_pil(image, face_info)
                resized_img = resize_image_pil(cropped_face, (resolution, resolution))
                # display(resized_img)
                # time.sleep(3)
                resized_img.save(output_path)

Creating images cropped&resized to 224 for VGG-Face:   0%|          | 0/70 [00:00<?, ?image/s]

Creating images cropped&resized to 220 for Facenet:   0%|          | 0/70 [00:00<?, ?image/s]

Creating images cropped&resized to 160 for Facenet512:   0%|          | 0/70 [00:00<?, ?image/s]

Creating images cropped&resized to 96 for OpenFace:   0%|          | 0/70 [00:00<?, ?image/s]

Creating images cropped&resized to 152 for DeepFace:   0%|          | 0/70 [00:00<?, ?image/s]

Creating images cropped&resized to 55 for DeepID:   0%|          | 0/70 [00:00<?, ?image/s]

Creating images cropped&resized to 112 for ArcFace:   0%|          | 0/70 [00:00<?, ?image/s]

Creating images cropped&resized to 112 for SFace:   0%|          | 0/70 [00:00<?, ?image/s]

In [ ]:
%%time
for model_name in model_names:
    resolution = model_resolutions[model_name]
    df_list = []

    files = glob.glob(os.path.join(folder_path, "**", "*.JPG"), recursive=True)

    # for img_path in tqdm(files, desc=f'Creating images cropped&resized to {resolution} for {model_name}', unit='image'):
    #     rel_path = os.path.relpath(img_path, folder_path)
    #     rel_path = rel_path.replace(os.path.sep, "_")
    #     output_path = os.path.join(tmp_folder, f"{rel_path[:-4]}_{resolution}.jpg")
    #     if not os.path.exists(output_path):
    #         img = cv2.imread(img_path)
    #         faces = DeepFace.extract_faces(img_path=img_path, detector_backend="retinaface")
    #         for face_info in faces:
    #             cropped_face = crop_to_square(img, face_info)
    #             resized_img = resize_image(cropped_face, (resolution, resolution))
    #             cv2.imwrite(output_path, resized_img)
           
    image_files = [f for f in os.listdir(tmp_folder) if f.endswith(f'_{resolution}.jpg')]
    
    for filename1 in tqdm(image_files, desc=f'Processing with {model_name}', unit='image'):
        img_path1 = os.path.join(tmp_folder, filename1)
        # img_1 = cv2.imread(img_path1)
        img_1 = Image.open(img_path1)
        img_1 = np.array(img_1)
        for filename2 in  tqdm(image_files, desc=f'Comparing all faces to {filename1}', unit='image'):
            img_path2 = os.path.join(tmp_folder, filename2)

            # img_2 = cv2.imread(img_path2)
            img_2 = Image.open(img_path2)
            img_2 = np.array(img_2)
            # https://stackoverflow.com/questions/74567500/tensorflow-unknownerror-graph-execution-error-jit-compilation-failed-op-in
            # with tf.device('/CPU:0'): 
            start_time = time.perf_counter()
            result = DeepFace.verify(
                img1_path=img_1,
                img2_path=img_2,
                enforce_detection=False,
                detector_backend="retinaface",
                model_name=model_name,
                distance_metric="euclidean_l2"
            )
            # print(f"time: {time.perf_counter() - start_time}")
            # print(f"distance for {img_path2}: {result['distance']}")
            img1_name = os.path.splitext(filename1)[0].rsplit('_', 1)[0]
            img2_name = os.path.splitext(filename2)[0].rsplit('_', 1)[0]

            df_list.append(pd.DataFrame({'img_1': [img1_name], 'img_2': [img2_name], 'distance': [result['distance']], 'verified': [result['verified']]}))
            
        df = pd.concat(df_list, ignore_index=True)
        
    csv_file_path = os.path.join(tmp_folder, f'{model_name}_comparison_results.csv')
    df.to_csv(csv_file_path, index=False)

Processing with VGG-Face:   0%|          | 0/70 [00:00<?, ?image/s]

Comparing all faces to aamartynenko_9_no_glasses_224.jpg:   0%|          | 0/70 [00:00<?, ?image/s]

Comparing all faces to aazheltov_1_no_glasses_224.jpg:   0%|          | 0/70 [00:00<?, ?image/s]

Comparing all faces to adletunov_no_glasses_224.jpg:   0%|          | 0/70 [00:00<?, ?image/s]

Comparing all faces to adpiskunov_no_glasses_224.jpg:   0%|          | 0/70 [00:00<?, ?image/s]

...

Comparing all faces to aazheltov_1_no_glasses_152.jpg:   0%|          | 0/70 [00:00<?, ?image/s]

Comparing all faces to adletunov_no_glasses_152.jpg:   0%|          | 0/70 [00:00<?, ?image/s]

Comparing all faces to adpiskunov_no_glasses_152.jpg:   0%|          | 0/70 [00:00<?, ?image/s]

Comparing all faces to aimilovanova_no_glasses_152.jpg:   0%|          | 0/70 [00:00<?, ?image/s]

Comparing all faces to ameliseenko_glasses_152.jpg:   0%|          | 0/70 [00:00<?, ?image/s]